# Acquiring and Processing Time Series Data
 
We are going to start working on the dataset we have chosen to work on throughout this book, process it in the right way, and learn about a few techniques for dealing with missing values.

In this chapter, we will cover the following topics:
- Understanding the time series dataset
- pandas datetime operations, indexing, and slicing – a refresher
- Handling missing data
- Mapping additional information
- Saving and loading files to disk
- Handling longer periods of missing data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
pio.kaleido.scope.mathjax = None
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
from itertools import cycle
%load_ext autoreload
%autoreload 2
np.random.seed()
tqdm.pandas()

C:\Users\dioni\AppData\Local\Temp\ipykernel_8216\996648543.py:11: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



In [3]:
os.makedirs("imgs/chapter_2", exist_ok=True)

-----

In [4]:
df = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00247/data_akbilgic.xlsx", skiprows=1)

df.head()

c:\Users\dioni\anaconda3\envs\modern-ts\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning:

Unknown extension is not supported and will be removed



,date,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,2009-01-05,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,2009-01-06,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,2009-01-07,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,2009-01-08,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,2009-01-09,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


Converting the date columns into pd.Timestamp/DatetimeIndex

In [5]:
pd.to_datetime("13-4-1987",dayfirst=True).strftime("%d, %B, %Y")


'13, April, 1987'

In [6]:
df['date'] = pd.to_datetime(df['date'], yearfirst=True)

In [7]:
df.date.dtype

dtype('<M8[ns]')

In [8]:
df.date.min(),df.date.max()

(Timestamp('2009-01-05 00:00:00'), Timestamp('2011-02-22 00:00:00'))

----

# Using the .dt accessor and datetime properties

Since the column is now in date format, all the semantic information that is encoded in the date can be
used through pandas datetime properties. We can access many datetime properties, such as month,
day_of_week, day_of_year, and so on, using the .dt accessor:


In [9]:
print(f"""
Date: {df.date.iloc[0]}
Day of year: {df.date.dt.day_of_year.iloc[0]}
Day of week: {df.date.dt.dayofweek.iloc[0]}
Week of Year: {df.date.dt.isocalendar().week[0]}
Month: {df.date.dt.month.iloc[0]}
Month Name: {df.date.dt.month_name().iloc[0]}
Quarter: {df.date.dt.quarter.iloc[0]}
Year: {df.date.dt.year.iloc[0]}
ISO Week: {df.date.dt.isocalendar().week.iloc[0]}
""")


Date: 2009-01-05 00:00:00
Day of year: 5
Day of week: 0
Week of Year: 2
Month: 1
Month Name: January
Quarter: 1
Year: 2009
ISO Week: 2



# Slicing and indexing

In [10]:
# Setting the index as the datatime column
df.set_index("date", inplace=True)

In [11]:
df["2010-01-04":]

,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
date,,,,,,,,,
2010-01-04,0.010229,0.014478,0.015916,0.000000,0.016018,0.000000,0.000000,0.016778,0.008399
2010-01-05,0.013898,0.024019,0.003111,-0.002722,0.004028,0.002535,0.002780,0.001316,0.008067
2010-01-06,0.007957,0.005706,0.000545,0.000409,0.001357,0.004635,0.006938,0.000586,0.005330
2010-01-07,0.007772,0.007498,0.003993,-0.002484,-0.000597,-0.004650,-0.003938,-0.000360,-0.006375
2010-01-08,-0.003189,0.000835,0.002878,0.003027,0.001356,0.010862,-0.002672,0.003554,0.001229
...,...,...,...,...,...,...,...,...,...
2011-02-16,0.008599,0.013400,0.006238,0.001925,0.007952,0.005717,0.018371,0.006975,0.003039
2011-02-17,0.009310,0.015977,0.003071,-0.001186,0.000345,0.002620,0.001686,-0.000581,0.001039
2011-02-18,0.000191,-0.001653,0.001923,0.002872,-0.000723,0.000568,0.005628,0.000572,0.006938


In [12]:
# Select all data between 2010-01-04 and 2010-02-06(not including)
df["2010-01-04": "2010-01-14"]

,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
date,,,,,,,,,
2010-01-04,0.010229,0.014478,0.015916,0.000000,0.016018,0.000000,0.000000,0.016778,0.008399
2010-01-05,0.013898,0.024019,0.003111,-0.002722,0.004028,0.002535,0.002780,0.001316,0.008067
2010-01-06,0.007957,0.005706,0.000545,0.000409,0.001357,0.004635,0.006938,0.000586,0.005330
2010-01-07,0.007772,0.007498,0.003993,-0.002484,-0.000597,-0.004650,-0.003938,-0.000360,-0.006375
2010-01-08,-0.003189,0.000835,0.002878,0.003027,0.001356,0.010862,-0.002672,0.003554,0.001229
2010-01-11,-0.016131,-0.006033,0.001745,0.000479,0.000704,0.007457,0.002417,-0.000498,0.005447
2010-01-12,-0.004549,-0.004339,-0.009425,-0.016273,-0.007140,0.000000,-0.005082,-0.009893,-0.003729
2010-01-13,0.017559,0.021362,0.008291,0.003383,-0.004593,-0.013335,0.004400,-0.000075,-0.007263
2010-01-14,0.002074,-0.002076,0.002424,0.004307,0.004503,0.015955,-0.008332,0.005500,0.002313


In [13]:
# Select data 2010 and before
df[: "2010"]

,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
date,,,,,,,,,
2009-01-05,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
2009-01-06,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2009-01-07,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
2009-01-08,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
2009-01-09,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802
...,...,...,...,...,...,...,...,...,...
2010-12-27,0.001346,0.000634,0.000000,0.000000,-0.002082,0.007444,0.000000,-0.005861,-0.002417
2010-12-28,0.001285,0.000438,0.000771,0.000197,0.000000,-0.006137,0.003504,0.000209,-0.000196
2010-12-29,0.004171,-0.005582,0.001009,0.003346,0.000000,0.005031,0.013300,0.001923,0.007861


In [14]:
# Select data between 2010-01 and 2010-06(both including)
df["2010-01": "2010-06"]

,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
date,,,,,,,,,
2010-01-04,0.010229,0.014478,0.015916,0.000000,0.016018,0.000000,0.000000,0.016778,0.008399
2010-01-05,0.013898,0.024019,0.003111,-0.002722,0.004028,0.002535,0.002780,0.001316,0.008067
2010-01-06,0.007957,0.005706,0.000545,0.000409,0.001357,0.004635,0.006938,0.000586,0.005330
2010-01-07,0.007772,0.007498,0.003993,-0.002484,-0.000597,-0.004650,-0.003938,-0.000360,-0.006375
2010-01-08,-0.003189,0.000835,0.002878,0.003027,0.001356,0.010862,-0.002672,0.003554,0.001229
...,...,...,...,...,...,...,...,...,...
2010-06-24,0.000166,-0.008024,-0.016946,-0.014455,-0.015236,0.000467,-0.018948,-0.017801,-0.006717
2010-06-25,-0.000920,-0.003591,0.002855,-0.007366,-0.010585,-0.019411,0.013778,-0.007954,-0.002311
2010-06-28,0.010132,0.016951,-0.002036,0.014168,0.004981,-0.004481,-0.009283,0.009125,0.000540


# Creating date sequences and managing date offsets 

Let’s look at a couple of ways of creating a sequence of dates:

## Specifying start and end dates with frequency

In [15]:
pd.date_range(start="2018-01-20", end="2018-01-23", freq="D").astype(str).tolist()

['2018-01-20', '2018-01-21', '2018-01-22', '2018-01-23']

## Specifying start and number of periods to generate in the given frequency

In [16]:
pd.date_range(start="2018-01-20", periods=4, freq="d").astype(str).tolist()

['2018-01-20', '2018-01-21', '2018-01-22', '2018-01-23']

##  Generating a date sequence with every 2 days

In [17]:
pd.date_range(start="2018-01-20", periods=4, freq="2d").astype(str).tolist()

['2018-01-20', '2018-01-22', '2018-01-24', '2018-01-26']

## Generating a date sequence every month. By default it starts with Month end

In [18]:
pd.date_range(start="2018-01-20", periods=4, freq="m").astype(str).tolist()

['2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30']

## Generating a date sequence every month, but month start

In [19]:
pd.date_range(start="2018-01-20", periods=4, freq="MS").astype(str).tolist()

['2018-02-01', '2018-03-01', '2018-04-01', '2018-05-01']

---- 

# Handling missing data

In [54]:
df = pd.read_csv("https://www.data.act.gov.au/resource/94a5-zqnn.csv", sep=",")
#Selecting one location and pm2.5
df.head(3)

,name,gps,datetime,no2,o3_1hr,o3_4hr,o3_8hr,co,pm10_1_hr,pm2_5_1_hr,...,aqi_co,aqi_no2,aqi_o3_1hr,aqi_o3_4hr,aqi_o3_8hr,aqi_pm10,aqi_pm2_5,aqi_site,date,time
0,Civic,"\n, \n(-35.285307, 149.131579)",2023-11-01T03:00:00.000,NaN,0.015,0.019,0.020,0.00,15.99,5.36,...,0.0,0.0,15.0,24.0,31.0,23.0,17.0,24.0,2023-11-01T00:00:00.000,3:00:00
1,Florey,"\n, \n(-35.220606, 149.043539)",2023-11-01T03:00:00.000,0.001,0.012,0.020,0.021,0.10,13.15,0.77,...,1.0,1.0,12.0,25.0,33.0,25.0,22.0,33.0,2023-11-01T00:00:00.000,3:00:00
2,Monash,"\n, \n(-35.418302, 149.094018)",2023-11-01T03:00:00.000,0.000,0.018,0.018,0.016,0.37,10.54,4.84,...,4.0,0.0,18.0,23.0,25.0,17.0,14.0,23.0,2023-11-01T00:00:00.000,3:00:00


In [55]:
df = df.loc[df.name=="Monash", ['datetime', 'pm2_5_1_hr']]
df.head(3)

,datetime,pm2_5_1_hr
2,2023-11-01T03:00:00.000,4.84
4,2023-11-01T02:00:00.000,3.89
8,2023-11-01T01:00:00.000,5.10


In [56]:
df.datetime = pd.to_datetime(df.datetime)

In [57]:
df.datetime.min(),df.datetime.max()

(Timestamp('2023-10-18 03:00:00'), Timestamp('2023-11-01 03:00:00'))

In [58]:
df.sort_values("datetime", inplace=True)
df.set_index("datetime", inplace=True)

In [59]:
df.loc["2023-10-18 7:00":"2023-10-18 09:00", "pm2_5_1_hr"] = np.nan

In [60]:
df.head()

,pm2_5_1_hr
datetime,
2023-10-18 03:00:00,2.12
2023-10-18 04:00:00,2.32
2023-10-18 05:00:00,1.86
2023-10-18 06:00:00,1.06
2023-10-18 07:00:00,NaN


In [61]:
def format_plot(fig, legends, font_size=15, title_font_size=20):
    names = cycle(legends)
    fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": title_font_size},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text="Value",
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text="Day",
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig

In [62]:
df_2 = df.loc["2023-10-18":"2023-10-18 20:00", "pm2_5_1_hr"].to_frame()
fig = px.line(df_2, x=df_2.index, y="pm2_5_1_hr", title="Missing Values in PM2.5")
fig = format_plot(fig, ["Original"])
fig.write_image("imgs/chapter_2/missing_values.png")
fig.show()

c:\Users\dioni\anaconda3\envs\modern-ts\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



## Last Observation Carried Forward (Forward Fill), Next Observation Carried Backward (Backward Fill) and Mean Value Fill

In [70]:
_df = df.copy()
_df = _df.loc["2023-10-18":"2023-10-18 20:00", "pm2_5_1_hr"].to_frame()
#Forward Fill
_df["ffill"] = _df['pm2_5_1_hr'].ffill()
# Backward Fill
_df["bfill"] = _df['pm2_5_1_hr'].bfill()
# Mean Value Fil
_df["mean_value"] = _df['pm2_5_1_hr'].fillna(df['pm2_5_1_hr'].mean())

In [71]:
#Plotting
plot_df = pd.melt(_df.reset_index(), id_vars="datetime", var_name="series")
fig = px.line(plot_df, x="datetime", y=["value"], line_dash="series", title="Forward, Backward, and Mean Value Fill")
fig = format_plot(fig, ['Original', 'Forward Fill', 'Backward Fill', "Mean Value Fill"])
fig.write_image("imgs/chapter_2/ffill_bfill_mean_missing_values.png")
fig.show()

c:\Users\dioni\anaconda3\envs\modern-ts\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [78]:
_df = df.copy()
_df = _df.loc["2023-10-18":"2023-10-18 20:00", "pm2_5_1_hr"].to_frame()
_df["Interpolate 'nearest'"] = _df['pm2_5_1_hr'].interpolate(method="nearest")
_df["Spline"] = _df['pm2_5_1_hr'].interpolate(method="spline", order=4)
_df["Polynomial"] = _df['pm2_5_1_hr'].interpolate(method="polynomial",
order=5)

In [79]:
#Plotting
plot_df = pd.melt(_df.reset_index(), id_vars="datetime", var_name="series")
fig = px.line(plot_df, x="datetime", y=["value"], line_dash="series", title="Forward, Backward, and Mean Value Fill")
fig = format_plot(fig, ['Original', 'Interpolate Nearest','Spline','Polynomial'])
fig.write_image("imgs/chapter_2/ffill_bfill_mean_missing_values.png")
fig.show()

c:\Users\dioni\anaconda3\envs\modern-ts\lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



----

# Converting the half-hourly block-level data (hhblock) into time series data

## Compact, expanded, and wide forms of data

There are many ways to format a time series dataset, especilly a dataset with many related time series, like the one we have now. Apart from the standard terminology of __wide__ data, we can also look at two non-standard ways of formatting time series data. 

Althoug there is no standard nomenclature for those, we will refer to them as __compact__ and __expanded__ in this book.

### Compact

Compact form data is when any particular time series occupies only a single row in the pandas DataFrame - this is, the time dimension is managed as an array within a DataFrame row. The time series identifiers and the metadata occupy the columns with scalar values and then the time series values; other time-varying features occupy the columns with an array. Two additional columns are inclued toextrapolate time `-start_datetime` and ´frequency´. If we know the start datetime and the frequency of the time series, we can easily construct the time and recover the time series from the DataFrame. This only works for regularly sampled time series. The advantage is that the DataFrames take up much less memory and are easy and faster to work with

![imange](https://i.imgur.com/ZfG2Sxg.png)


### Expanded form

The expanded form is when the time series is expanded along the rows of a DataFrame. If there are $n$ steps in the time series, it occupies "n" rows in the DataFrame. The time series identifiers and the metadata get repeated along all the rows. The time-varying features also get expanded along the rows. And instead of the start date and frequency, we have the datatime as a column:

![extended form](https://i.imgur.com/wgzPLco.png)

